# Projeto 2 - Ciência dos Dados

**Nome: Thiago Maitan Pegorer**

**Nome: Raphael Jacob Butori**

## Classificador automático de sentimento

### Preparando o ambiente no jupyter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [22]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle


## Autenticando no Twitter

In [4]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @pegorer_thi

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

# Projeto

### Escolha de um produto e coleta das mensagens

In [5]:
#Produto escolhido:
produto = 'Samsung'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

In [6]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't':
        msgs.append(msg.full_text.lower())
        i += 1
        if i > n:
            break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)


In [7]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

In [14]:
tw=pd.read_excel("Samsung.xlsx")
tw.head()

,Treinamento,Classificador
0,alerta de oferta: samsung galaxy a30 a partir ...,0
1,apesar de amar a samsung. vi hoje que a #xiaom...,0
2,odeio samsung. pqppp,1
3,@hearthstoneness @cissamagazine na loja tbm né...,0
4,tenho que comprar um celular novo mas 0 vontad...,1


### Limpando pontuação do excel

In [36]:
import string

pontuacao = string.punctuation

#lista para depois tirar os pontos
po = ["\n", "\t"]

limpeza = []

#dar espaço nos emojis e limpar os pontos
for tweet in tw["Treinamento"]:
    t = ''
    for palavra in tweet:
        if palavra in po:
            t += " "
        elif palavra not in pontuacao:
            t += palavra
    limpeza.append(t)

tw_sam =  pd.DataFrame()
tw_sam['Treinamento'] = limpeza
tw_sam['Classificador'] = tw['Classificador']

tw_sam.head()



,Treinamento,Classificador
0,alerta de oferta samsung galaxy a30 a partir d...,0
1,apesar de amar a samsung vi hoje que a xiaomi ...,0
2,odeio samsung pqppp,1
3,hearthstoneness cissamagazine na loja tbm né n...,0
4,tenho que comprar um celular novo mas 0 vontad...,1


In [47]:
relevante=tw_sam.loc[tw_sam.Classificador==1]
irelevante=tw_sam.loc[tw_sam.Classificador==0]